# Affective Computing - Final Project
- Facial Expression Recognition with TensorFlow

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import pandas as pd
import cv2

<br><br>

## Introduction

- The dataset is available here
    - https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

In [3]:
df = pd.read_csv('drive/My Drive/Faks/Afektivno/projekt/data/fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


- Directory structures where images will be saved to their respective folders:

In [4]:
%mkdir images
%cd images
%mkdir train test validation
%cd train
%mkdir Angry Disgust Fear Happy Sad Surprise Neutral
%cd ..
%cd test 
%mkdir Angry Disgust Fear Happy Sad Surprise Neutral
%cd ..
%cd validation 
%mkdir Angry Disgust Fear Happy Sad Surprise Neutral
%cd ..
%cd ..

/content/images
/content/images/train
/content/images
/content/images/test
/content/images
/content/images/validation
/content/images
/content


- This will be great for train, test, and validation:

In [5]:
df['Usage'].value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

- Multi-class classification problem:

In [ ]:
classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

- Classes are disballanced:

In [7]:
df['emotion'].value_counts()

3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64

<br><br>

## Data Preparation
- Each row of PIXELS attribute consists of a string representing pixels
    - Has 2304 elements (48x48)
- Will get converted to list of ints and reshaped to 2D array
- Then using OpenCV the images are saved to their folders:

In [ ]:
def process_image(str_pixels):
    pixels = str_pixels.split()
    pixels = [int(pixel) for pixel in pixels]
    pixels = np.array(pixels)
    pixels = pixels.reshape(48, 48)
    return pixels

In [9]:
i = 0
for row in df.itertuples(index=False):
    i += 1
    if i % 1000 == 0:
        print(f'Processed {i} images...')
    
    img = process_image(row.pixels)
    curr_emotion = classes[row.emotion]

    if row.Usage == 'Training':
        cv2.imwrite(f'/content/images/train/{curr_emotion}/{i}.jpg', img)
    elif row.Usage == 'PublicTest':
        cv2.imwrite(f'/content/images/test/{curr_emotion}/{i}.jpg', img)
    else:
        cv2.imwrite(f'/content/images/validation/{curr_emotion}/{i}.jpg', img)

Processed 1000 images...
Processed 2000 images...
Processed 3000 images...
Processed 4000 images...
Processed 5000 images...
Processed 6000 images...
Processed 7000 images...
Processed 8000 images...
Processed 9000 images...
Processed 10000 images...
Processed 11000 images...
Processed 12000 images...
Processed 13000 images...
Processed 14000 images...
Processed 15000 images...
Processed 16000 images...
Processed 17000 images...
Processed 18000 images...
Processed 19000 images...
Processed 20000 images...
Processed 21000 images...
Processed 22000 images...
Processed 23000 images...
Processed 24000 images...
Processed 25000 images...
Processed 26000 images...
Processed 27000 images...
Processed 28000 images...
Processed 29000 images...
Processed 30000 images...
Processed 31000 images...
Processed 32000 images...
Processed 33000 images...
Processed 34000 images...
Processed 35000 images...


<br><br>

## Model Definition

In [ ]:
import os
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

- We have 7 classes of 48x48 images
- After some testing batch size of 64 seems to be working pretty well

In [ ]:
num_classes = 7
img_rows, img_cols = 48, 48
batch_size = 64

In [ ]:
train_data_dir = '/content/images/train'
validation_data_dir = '/content/images/validation'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


- Here is the model
- Tried over 15 architectures and this one yields the best accuracy on the test set
- Transfer learning approach didn't get over 40% accuracy

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(4096, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(2048, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes, kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

- Will also define some callbacks
    - Early stopping
    - Reduce learning rate on plateau
- Model is trained for 50 epochs

In [17]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    'FERModel.h5',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=9,
    verbose=1,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_delta=0.0001
)

callbacks = [earlystop, checkpoint, reduce_lr]
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.001),
    metrics=['accuracy']
)

nb_train_samples = 28709
nb_validation_samples = 3589
epochs = 50

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
447/448 [============================>.] - ETA: 0s - loss: 2.3540 - accuracy: 0.1802
Epoch 00001: val_loss improved from inf to 1.80696, saving model to FERModel.h5
448/448 [==============================] - 20s 44ms/step - loss: 2.3530 - accuracy: 0.1804 - val_loss: 1.8070 - val_accuracy: 0.2419 - lr: 0.0010
Epoch 2/50
448/448 [==============================] - ETA: 0s - loss: 1.8825 - accuracy: 0.2184
Epoch 00002: val_loss improved from 1.80696 to 1.77626, saving model to FERModel.h5
448/448 [==============================] - 19s 42ms/step - loss: 1.8825 - accuracy: 0.2184 - val_loss: 1.7763 - val_accuracy: 0.2567 - lr: 0.0010
Epoch 3/50
447/448 [============================>.] - ETA: 0s - loss: 1.8050 - accuracy: 0.2493
Epoch 00003: val_loss improved from 1.77626 to 1.76356, saving model to FERModel.h5
448/448 [==============================] - 19s 42ms/step - loss: 1.8049 - accuracy: 0.2495 - val

<br><br>

## Performance Testing

- The model will now be evaluated on previously unseen data (only train and valid sets were visible in the training process)

In [18]:
test_data_dir = '/content/images/test/'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3589 images belonging to 7 classes.


In [19]:
results = model.evaluate_generator(test_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [ ]:
test_loss, test_acc = results

- Obtained 63% accuracy on the train set
- Dummy model (predicts random classes) would get around 14%

In [21]:
test_acc

0.630259096622467